In [0]:
import numpy as np
#os hago el trabajo más duro

Implementación de un modelo de Sentiment Analysis con algoritmo de Machine Learning Clásico.

Se prepara y filtra el dataset antes del entrenamiento  para un algoritmo de clasificacion con datos etiquetados con entrenamiento supervisado.


In [2]:
# Se carga el dataset para la práctica desde un xlsx (no tengo la versión buena de excel, y por no tanto no puedo
# recortar documentos tipo csv).

import openpyxl

import numpy as np 
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# Crea un dataframe con las columnas :
# ItemID : número de registro
# Sentiment : 0 o 1 (negativo o positivo)
# SentimentText : texto
#doc = pd.read_excel('/content/drive/My Drive/BootCamp - BigDataIV - NLP/data_practica_train_sentiment _1000.xlsx')
doc = pd.read_excel('/content/drive/My Drive/BootCamp - BigDataIV - NLP/data_practica_train_sentiment.xlsx')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
doc

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...
...,...,...,...
99984,99996,0,@Cupcake seems like a repeating problem hop...
99985,99997,1,@cupcake__ arrrr we both replied to each other...
99986,99998,0,@CuPcAkE_2120 ya i thought so
99987,99999,1,@Cupcake_Dollie Yes. Yes. I'm glad you had mor...


In [0]:
# Descartamos del dataset el ItemID, pues el número de fila no lo usamos para nada
doc = doc.drop(['ItemID'], axis=1)

In [0]:
# Invertimos el orden de las columnas, para crear unigramas
cols = doc.columns.tolist()
cols.insert(0, cols.pop(-1))
doc = doc[cols]

In [6]:
#Pidiendo información, vemos que hay nulos en la columna de texto.
doc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99989 entries, 0 to 99988
Data columns (total 2 columns):
SentimentText    99860 non-null object
Sentiment        99989 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [0]:
# Eliminamos las filas con nulos en el campo texto 
doc=doc.dropna()

In [0]:
# Sustituimos los valores del campo texto 0 por KO y 1 por OK, para facilitar el proceso 
doc.Sentiment = doc.Sentiment.replace({0 : "ko", 1 : "ok"})

In [9]:
# Comprobamos cuántos KO y OK hay
doc.Sentiment.value_counts()

ok    56405
ko    43455
Name: Sentiment, dtype: int64

In [0]:
# Convertimos el DF a lista
datos=doc.values.tolist()
datos1=datos

Usando Bayes (podemos asumir que los comentarios del Dataset son independientes unos de otros).

In [0]:
# Creamos un diccionario con las frases del texto que corresponden a cada clase
def prepare_data(data):  # De entrada los registros del dataset 
    data_together = {}
    for x, y in data:    # para cada clase de Sentiment añadira el SentimentText 
        if y in data_together:
            data_together[y].append(x)
        else: 
            data_together[y]=[x]
    return data_together 

In [0]:
# Cálculo de la probabilidad que tiene cada clase 
from collections import Counter

def get_prior(data):   # De entrada los registros del dataset 
    prior = Counter([y for _, y in data])  # Contador de frases correspondientes a cada clase ('ok' y 'ko')
    p = {}
    for pr in prior.keys():
        p[pr]=prior[pr]/len(data) # Calcula la probabilidad que tiene cada clase en el total
    return p

In [0]:
def get_likelihood(data):
    tokens_in_class = {}
    d = prepare_data(data)  # Usamos los pares generados
    #cuantos tokens hay en cada clase (hay dos clases : 'ok' y 'ko')
    for label, strings in d.items():
        tokens_in_class[label] = Counter([token for string in strings for token in string.split(" ")])
        print(tokens_in_class[label].most_common(5)) # muestra los 5 unigrams más repetidos por clase

    tokens_in_class['ok'].pop('') # elimina clase '' que no aporta nada
    tokens_in_class['ko'].pop('')

    print(tokens_in_class[label].most_common(5)) # muestra los 5 unigrams más repetidos por clase

    print(tokens_in_class['ok'][''])

    # cálculo número de tokens por clase
    tokens_per_class = {}
    for label in tokens_in_class.keys():
        tokens_per_class[label] = sum(tokens_in_class[label].values())
    print(tokens_per_class)   # muestra por pantalla cuantos tokens hay por clase

    print(tokens_in_class['ok'][''])

    # probabilidad de cada token en cada clase
    likelihood = {}
    for label in tokens_in_class.keys():
        likelihood[label] = {}
        for token in tokens_in_class[label].keys():
            likelihood[label][token] = tokens_in_class[label][token]/tokens_per_class[label]

    print(tokens_in_class['ok'][''])

    return likelihood    

In [0]:
# Entrenamos el modelo
def train(data):
    likelihood = get_likelihood(data) # probabilidad de token por clase
    prior = get_prior(data)           # probabilidad de clase 
    return likelihood, prior

In [15]:
model = train(datos) # las dos primeras líneas muestran los 5 tokens más repetidos por clase
                     # la tercera, el número de tokens por clase ('ok' y 'ko') 

[('', 54066), ('I', 17616), ('to', 14154), ('the', 11986), ('i', 9547)]
[('', 69827), ('the', 16049), ('I', 15216), ('to', 14616), ('you', 14012)]
[('the', 16049), ('I', 15216), ('to', 14616), ('you', 14012), ('a', 12623)]
0
{'ko': 603688, 'ok': 724967}
0
0


In [0]:
from math import log2, log, exp

# Cálculo de la funcíon exponencial normalizada
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

In [0]:
# Modelo de predicción de a qué clase ('ok' o 'ko') pertenece un SentimentText
# Muestra por pantalla el SentimentText y la probabilidad que pertenezca a la clase 'ko', o a la 'ok'
def predict(model, data, alpha=0.00000001): # usamos el modelo anterior entrenado, el Dataset de la práctica 
    likelihood, prior = model               # probabilidad de cada token
    classes = list(prior.keys())
    print(classes)                          # ['ko', 'ok']
    
    # matriz de predicciones :  con probabilidades muy pequeñas, usamos log, convirtiendo las probabilidades una
    # a una y luego usar la inversa.
    predicted = np.zeros((len(data),len(classes))) # array a ceros de dos columnas y num registros como filas 
    for i, doc in enumerate(data):  # para cada palabra
        doc_rep = doc.split(" ")
        probs = np.zeros((len(classes)))
        for j, label in enumerate(classes): # para cada clase
            pwc =[log(likelihood[label][x]) if x in likelihood[label] else log(alpha) for x in doc_rep] 
            #sumar en log es igual a multiplicar normal.
            pwc = np.array(pwc)
            prob = np.sum(pwc)
            probs[j] = prob+log(prior[label])
        predicted[i,:] = softmax(probs)
    return np.round(predicted, 2)

In [0]:
 xs = [x for x, _ in datos]  # Nos quedamos con el SentimentText en forma de array

In [19]:
# Hacemos una prueba con frases cortas 
datos_ej = ['this is shit', 'I hate this city', 'I like Madrid', 'smile today and tomorrow', 'war is peace', 
         'freedom is slavery', 'ignorance is strenght', 'flowers are beautiful', 'make America great again']
         
predictions = predict(model, datos_ej)     # De entrada, el modelo entrenado arriba y el SentimentText. Devuelve la 
                                           # probabilidad de que cada token pertenezca a cada clase
for datap, p in zip(datos_ej, predictions): # A cada SentimentText damos su probabilidad de pertenencer a cada clase
    print(datap, p)

# Los resultados siguientes han sido probados con 1000 registros del dataset de entrada. Como se puede ver, la 
# predicción es pobre.

#                         ['ko', 'ok']
#this is shit             [1. 0.]
#I hate this city         [0. 1.]
#I like Madrid            [0.74 0.26] --> vaya hombre
#smile today and tomorrow [0.35 0.65] --> no le convence Ned Flanders
#war is peace             [1. 0.]     --> no se ha leído a Orwell
#freedom is slavery       [0.69 0.31] --> preocupante que esté totalmente convencido
#ignorance is strenght    [0.69 0.31] --> jo!
#flowers are beautiful    [0. 1.]     --> esta era fácil
#make America great again [0.87 0.13] --> espero que Trump no vea ésto


# Los resultados siguientes han sido probados con 1000 registros del dataset de entrada, y además, se eliminó la
# clase ''. Cambia levemente.

#                         ['ko', 'ok']
#this is shit             [1. 0.]
#I hate this city         [0. 1.]
#I like Madrid            [0.75 0.25]
#smile today and tomorrow [0.38 0.62]
#war is peace             [1. 0.]
#freedom is slavery       [0.7 0.3]
#ignorance is strenght    [0.7 0.3]
#flowers are beautiful    [0. 1.]
#make America great again [0.88 0.12]


# Esta prueba se hace con todos los registros del dataset de entrada(los 10000). Los resultados mejoran, pero aún
# así hay lagunas.
#                         ['ko', 'ok']
#this is shit             [0.67 0.33]  --> puede que decir que es una m no sea del todo malo
#I hate this city         [0.88 0.12]  --> quizá odie la ciudad con razón?
#I like Madrid            [0.49 0.51]  --> todavía no le convence
#smile today and tomorrow [0.19 0.81]  --> mejora
#war is peace             [0.15 0.85]  --> menos mal que cambió de opinión
#freedom is slavery       [0.48 0.52]  --> vaya por Dios
#ignorance is strenght    [0.74 0.26]  --> otro miembro de Ingsoc
#flowers are beautiful    [0.07 0.93]  --> alguna flor no es bella
#make America great again [0.17 0.83]  --> cambión el voto para Donald


['ko', 'ok']
this is shit [0.67 0.33]
I hate this city [0.88 0.12]
I like Madrid [0.49 0.51]
smile today and tomorrow [0.19 0.81]
war is peace [0.15 0.85]
freedom is slavery [0.48 0.52]
ignorance is strenght [0.74 0.26]
flowers are beautiful [0.07 0.93]
make America great again [0.17 0.83]


In [20]:
import string
printable_chars = string.printable  # Muestra los caracteres
printable_chars

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [21]:
# Filtra las frases de entrada según los caracteres anteriores
datos_w = [(''.join([c if c in printable_chars else '' for c in d[0]]), d[1]) for d in datos1]
datos_w

[('                     is so sad for my APL friend.............', 'ko'),
 ('                   I missed the New Moon trailer...', 'ko'),
 ('              omg its already 7:30 :O', 'ok'),
 ("          .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)...",
  'ko'),
 ('         i think mi bf is cheating on me!!!       T_T', 'ko'),
 ('         or i just worry too much?        ', 'ko'),
 ('       Juuuuuuuuuuuuuuuuussssst Chillin!!', 'ok'),
 ('       Sunny Again        Work Tomorrow  :-|       TV Tonight', 'ko'),
 ('      handed in my uniform today . i miss you already', 'ok'),
 ('      hmmmm.... i wonder how she my number @-)', 'ok'),
 ('      I must think about positive..', 'ko'),
 ('      thanks to all the haters up in my face all day! 112-102', 'ok'),
 ('      this weekend has sucked so far', 'ko'),
 ('     jb isnt showing in australia any more!', 'ko'),
 ('     ok thats it you win.', 'ko'),
 ('    &lt;-------- This i

In [22]:
# Se fijan los parámetros 
high = len(datos_w) # número de registros del dataset 99.860
test_size = 0.2     # se toma el 20% para el test
size = int(high*test_size) # 19.972 registros para el test
idx = np.arange(len(datos_w))  # array con el número de índices
print('Todos los elementos: {}'.format(idx)) # Muestra todos los elementos del dataset indexados

#random sampling 
test_idx = np.random.choice(idx, size=size, replace=False) #Selecciona aleatoriamente elementos para el test

#masking test indices, rest train 
train_mask = np.ones(len(idx), dtype=bool)
train_mask[test_idx] = False
train_idx = idx[train_mask==True]
print('train idx: {}'.format(train_idx)) # muestra los índices elegidos para el train
print('test idx: {}'.format(test_idx))   # muestra los índices elegidos para el test

Todos los elementos: [    0     1     2 ... 99857 99858 99859]
train idx: [    0     1     2 ... 99856 99857 99858]
test idx: [66512 54108 23530 ...  9313 87917   447]


In [0]:
train_data = np.array(datos_w)[train_idx].tolist() # convierte los registros de train en lista
test_data = np.array(datos_w)[test_idx].tolist() # convierte los registros de test en lista

In [0]:
from collections import Counter

label_distribution = Counter([y for _, y in datos_w]) # contador de reg cada clase totales :  'ko' 43455 'ok' 56405 
label_train = Counter([y for _, y in train_data])     # contador de reg cada clase en train : 'ko' 34792 'ok' 45096
label_test = Counter([y for _, y in test_data])       # contador de reg cada clase en test :  'ko' 8863 'ok' 11309

s_total = pd.Series([y for _, y in datos_w])          # clases de cada registro en totales
s_train = pd.Series([y for _, y in train_data])       # clases de cada registro en train
s_test = pd.Series([y for _, y in test_data])         # clases de cada registro en test

df = pd.DataFrame([s_total, s_train, s_test]).T       # convertir los campos anteriores en dataframe
df.rename(index=str, columns={0: 'total', 1: 'train', 2:'test'}, inplace=True)

In [0]:
from sklearn.model_selection import StratifiedShuffleSplit

# Entrenamiento del dataset de train
def split_train_val_test(dataset, split=0.2):

    x, y = zip(*dataset)  # Separa en SentimentText, y Sentiment(ok/ko)
    x = np.array(list(x)) # Convierte SentimentText en array
    y = np.array(list(y)) # Convierte Sentiment en array

# Este objeto de validación cruzada devuelve pliegues aleatorios estratificados. Los pliegues se realizan
# conservando el porcentaje de muestras para cada clase.
    sss = StratifiedShuffleSplit(n_splits=1, test_size=split, random_state=1337) #l33t seed
    for train_index, test_index in sss.split(x, y): #divide el resultado en conjunto de test y train indexados
        x_train, x_val = x[train_index], x[test_index]
        y_train, y_val = y[train_index], y[test_index]
    splits = {'train':(x_train, y_train), 'test':(x_val, y_val)} # crea un array con cada SentimentText por clase
    return splits

In [0]:
split = split_train_val_test(datos_w)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV  # Se le pasa un modelo y devuelve parámetros para le algortimo
from sklearn.pipeline import Pipeline    # Diseña una serie de pasos para optimizar las características de la 
                                         # extracción de features                                          
from sklearn.feature_extraction.text import TfidfTransformer

parameters = {                                      # parámetros para el gridsearch la pipeline en orden
    'vect__max_df': (0.5, 0.75, 1.0),               # elimina términos que aparecen demasiadas veces 
    'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),  # unigrams o bigrams o trigrams
}

# Define a pipeline combining a text feature extractor with a simple # classifier
pipeline = Pipeline([                     # Pasos para la ejecución del algoritmo
  ('vect', CountVectorizer(max_df=0.5)),  # Convierte colección de documentos de texto a una matriz contador tokens
  ('clf', MultinomialNB()),               # Clasificador de Bayes para modelos multinomiales
])

# pipeline.get_params()      parámetros a mirar en el gridsearch
# vect__analyzer : 'word'    tokeniza por palabras
# vect__binary : False       devuelve T/F
# vect__lowercase : True     toda la frase en minúsculas
# vect__max_features : None  número max de características que permite que se extraigan (ninguno)
# vect__stop_words : None    no usar stop_words

## 1.Sentiment Analysis

In [0]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

In [29]:
# Nos devolverá los mejores parámetros para el modelo, incluyendo el extractor de features
grid_search.fit(split['train'][0], split['train'][1])
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
print("Best score: %0.3f" % grid_search.best_score_)

# vect__max_df: 0.5
#	vect__ngram_range: (1, 2)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  3.4min finished


	vect__max_df: 0.5
	vect__ngram_range: (1, 2)
Best score: 0.770


In [30]:
 # Le pasamos al pipeline los mejores parámetros extraídos antes
 pipeline.set_params(**best_parameters)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.5,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [31]:
# nos devuelve el score para el test
predictions = pipeline.score(split['test'][0], split['test'][1])
print('TEST SCORE: {}'.format(predictions))

# TEST SCORE: 0.7738834368115362 (no es el mejor)

TEST SCORE: 0.7738834368115362


In [32]:
predictions = pipeline.predict(datos_ej) # aplica transformaciones a datos y predice con el estimador final
predictions_probs = pipeline.predict_proba(datos_ej) # aplica trasnformaciones y predice la prob del estimador final
prediction_hand = predict(model, datos_ej)
# Muestra por pantalla la frase, la clase predecida, la probabilidad de que pertenezcan a las clases, y predicción
# de la probabilidad
for x, y, p, ph in zip(datos_ej, predictions, predictions_probs, prediction_hand):
    print('{} ------> {} {} {}'.format(x, y, p, ph))

# Comparando con los resultados obtenidos arriba (los de prediction_hand):
#                          ['ko', 'ok']
# this is shit             [0.67 0.33]  --> puede que decir que es una m no sea del todo malo
# I hate this city         [0.88 0.12]  --> quizá odie la ciudad con razón?
# I like Madrid            [0.49 0.51]  --> todavía no le convence
# smile today and tomorrow [0.19 0.81]  --> mejora
# war is peace             [0.15 0.85]  --> menos mal que cambió de opinión
# freedom is slavery       [0.48 0.52]  --> vaya por Dios
# ignorance is strenght    [0.74 0.26]  --> otro miembro de Ingsoc
# flowers are beautiful    [0.07 0.93]  --> alguna flor no es bella
# make America great again [0.17 0.83]  --> cambión el voto para Donald

# Vemos que mejora en todas las frases, excepto en :
# ignorance is strenght ------> ko [0.5513334 0.4486666] [0.74 0.26]
# quizá demasiado surrealista para un algoritmo

['ko', 'ok']
this is shit ------> ko [0.77601521 0.22398479] [0.67 0.33]
I hate this city ------> ko [0.97759626 0.02240374] [0.88 0.12]
I like Madrid ------> ok [0.43033853 0.56966147] [0.49 0.51]
smile today and tomorrow ------> ok [0.16091688 0.83908312] [0.19 0.81]
war is peace ------> ok [0.10135486 0.89864514] [0.15 0.85]
freedom is slavery ------> ko [0.54348781 0.45651219] [0.48 0.52]
ignorance is strenght ------> ko [0.5513334 0.4486666] [0.74 0.26]
flowers are beautiful ------> ok [0.0370839 0.9629161] [0.07 0.93]
make America great again ------> ok [0.16497639 0.83502361] [0.17 0.83]
